## First try at getting some data

In [7]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [8]:
#Essentials
import numpy as np
import pandas as pd
import pickle
import re
import datetime as dt
import time as t
from termcolor import colored

#SQL related
import psycopg2
import sqlite3
from sqlalchemy import create_engine
import pandas.io.sql as pd_sql

#Visualization
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
from  statsmodels.graphics.tsaplots import plot_pacf

%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 25, 6
rcParams.update({'font.size': 14})

### Load all the data into an SQL database

In [9]:
# Set up the database connection
conn = psycopg2.connect(dbname="taxi_rides_db", user="auste_m")

In [10]:
# taxi_rides_query = """SELECT * FROM taxi_rides WHERE trip_start_timestamp BETWEEN '2016-02-01' AND '2016-02-15';""" 
# weather_query = """SELECT * FROM chicago_weather WHERE date BETWEEN '2016-02-01' AND '2016-02-15';"""

taxi_weather_query = """SELECT TR.trip_start_timestamp,
                                TR.trip_end_timestamp,
                                TR.trip_seconds,
                                TR.trip_miles,
                                CASE WHEN TR.tolls IS NULL THEN TR.fare ELSE TR.fare + TR.tolls END as fare_with_tolls,
                                TR.pickup_centroid_latitude,
                                TR.pickup_centroid_longitude,
                                TR.pickup_centroid_location,
                                TR.dropoff_centroid_latitude,
                                TR.dropoff_centroid_longitude,
                                TR.dropoff_centroid_location,
                                CW.avg_daily_wind_speed,
                                CW.avg_temp_f,
                                CASE WHEN CW.fog = 1 OR CW.heavy_fog = 1 OR ice_fog = 1 OR mist = 1 THEN 1 ELSE 0 END as fog,
                                CASE WHEN CW.rain = 1 OR CW.drizzle = 1 OR CW.hail = 1 THEN 1 ELSE 0 END as rain,
                                CASE WHEN CW.snow = 1 OR CW.freezing_rain = 1 OR drifting_snow = 1 THEN 1 ELSE 0 END as snow
                                
                        FROM taxi_rides_clean as TR
                        LEFT JOIN chicago_weather as CW
                        ON CAST(TR.trip_start_timestamp AS DATE) = CAST(date AS DATE)
                        WHERE trip_start_timestamp BETWEEN '2016-01-01' AND '2016-12-31';""" 

In [11]:
taxi_weather_df = pd.read_sql_query(taxi_weather_query, con=conn)

KeyboardInterrupt: 

In [ ]:
taxi_weather_df.describe()

> Original number of records for 1st-15th February 2016: **831,790**  
> Original number of records for the whole of 2016: **19,846,602**

In [ ]:
print('Total number of taxi trips (in mln):', str(round(taxi_weather_df['fare_with_tolls'].count() / 10**6)))
print('Total $ spent on taxi trips (in mln):', str(round(taxi_weather_df['fare_with_tolls'].sum() / 10**6)))
print('Total number of years of taxi trips:', str(round(taxi_weather_df['trip_seconds'].sum() / 60 / 60 / 24 / 365, 0)))

In [ ]:
#Let's explore the dataset
null_records = taxi_weather_df[(taxi_weather_df['pickup_centroid_latitude'].isnull()) | (taxi_weather_df['pickup_centroid_longitude'].isnull()) | 
                (taxi_weather_df['dropoff_centroid_latitude'].isnull()) | (taxi_weather_df['dropoff_centroid_longitude'].isnull()) | 
                (taxi_weather_df['trip_seconds'].isnull()) | (taxi_weather_df['trip_miles'].isnull()) | 
                (taxi_weather_df['fare_with_tolls'].isnull())]['trip_start_timestamp'].count()

print(f'There are {null_records} records with a null value in at least one of the fields mentioned above.')

## Sweeping time!

In [ ]:
taxi_weather_df.head()

In [ ]:
#Change payment type to boolean
taxi_weather_df['payment_type'] = pd.Categorical(taxi_weather_df['payment_type'])

In [ ]:
#Let's drop all the records with a missing lat / long in either pickup or dropoff
taxi_weather_clean = taxi_weather_df.dropna(subset=['pickup_centroid_latitude', 'pickup_centroid_longitude', 'dropoff_centroid_latitude'
                                                    , 'dropoff_centroid_longitude', 'trip_seconds', 'trip_miles', 'fare_with_tolls'])

In [ ]:
taxi_weather_clean.head(2)

In [ ]:
taxi_weather_clean[['fog', 'rain', 'snow']] = taxi_weather_clean[['fog', 'rain', 'snow']].fillna(value=0)

In [ ]:
taxi_weather_clean.head()

In [ ]:
taxi_weather_clean.count()[0]

> Remaining number of records after removing missing values containing records for 1st-15th February 2016: **81%** (673,474)  
> Remaining number of records after removing missing values containing records for whole 2016: **84%** (16,656,816)

In [ ]:
#Let's get rid of the zeroes
taxi_weather_clean = taxi_weather_clean[(taxi_weather_clean['fare_with_tolls'] != 0.0) 
                                          & (taxi_weather_clean['trip_seconds'] != 0.0) 
                                          & (taxi_weather_clean['trip_miles'] != 0.0)]

In [ ]:
taxi_weather_clean.count()[0]

> Remaining number of records after null and zero value removal for 1st-15th February 2016: **64%** (533,775)   
> Remaining number of records after null and zero value removal for the whole 2016: **67%** (13,354,486)

In [ ]:
#Let's have a look if the miles per minute look reasonable
plt.figure(dpi=100)
plt.scatter(taxi_weather_clean['trip_miles'], taxi_weather_clean['trip_seconds'] / 3600.0)
plt.title('Miles per hour')
plt.xlabel('Miles')
plt.ylabel('Hours');

In [ ]:
miles_per_hour = taxi_weather_clean['trip_miles'] / (taxi_weather_clean['trip_seconds'] / 3600.0)

In [ ]:
plt.figure(dpi=100)
miles_per_hour.hist(bins=10000)
plt.xlim(xmin=0.0, xmax=100.0)
plt.title('Miles per hour histogram')
plt.xlabel('Miles / hour')
plt.ylabel('Frequency');

In [ ]:
#Removing clear error records from the dataset (e.g. $1,000 for 0.1 mile and 6 min ride). Settling at max 100 miles an hour
taxi_weather_clean = taxi_weather_clean[(taxi_weather_clean['trip_miles'] / (taxi_weather_clean['trip_seconds'] / 3600.0)) <= 100.0]

In [ ]:
taxi_weather_clean.count()[0]

> Number of records excluded by removing instances with speed higher than 100: **%** ()  
> Number of records remaining (whole 2016): **67%** (13,301,862)

In [ ]:
#Let's have a look if the miles per hour look reasonable
plt.figure(dpi=100)
plt.scatter(taxi_weather_clean['trip_miles'], taxi_weather_clean['trip_seconds'] / 3600.0)
plt.title('Miles per hour (after cleaning)')
plt.xlabel('Miles')
plt.ylabel('Hours');

In [ ]:
#Let's see that's the distribution of $/mile to better identify incorrect values
fare_per_mile = taxi_weather_clean['fare_with_tolls'] / taxi_weather_clean['trip_miles']
fare_per_minute = taxi_weather_clean['fare_with_tolls'] / (taxi_weather_clean['trip_seconds'] / 60)

#Removing obvious outliers
outlier_indices = list((fare_per_mile[fare_per_mile > 5000]).index)
fare_per_mile = fare_per_mile.drop(index=outlier_indices)
fare_per_minute = fare_per_minute.drop(index=outlier_indices)

plt.figure(dpi=100)
plt.scatter(fare_per_mile, fare_per_minute)
plt.title('Fare per mile versus fare per minute')
plt.xlabel('Fare/Mile')
plt.ylabel('Fare/Minute');

In [ ]:
#Based on above analysis, I'm setting a max fare/mile at $250 and max fare/minute to $50
taxi_weather_clean = taxi_weather_clean[((taxi_weather_clean['fare_with_tolls'] / taxi_weather_clean['trip_miles']) <= 50) &
                                 ((taxi_weather_clean['fare_with_tolls'] / (taxi_weather_clean['trip_seconds'] / 60.0)) <= 250)]

In [ ]:
taxi_weather_clean.count()[0]

> Number of records excluded by removing instances of fare/mile of more than USD 250 and fare/minute of more than USD 50: **8%** (39,999)  
> Number of records remaining: **62%** (12,343,799)

In [ ]:
#Let's see that's the distribution of $/mile to better identify incorrect values
fare_per_mile = taxi_weather_clean['fare_with_tolls'] / taxi_weather_clean['trip_miles']
fare_per_minute = taxi_weather_clean['fare_with_tolls'] / (taxi_weather_clean['trip_seconds'] / 60)


#Replot to see the effect
plt.figure(dpi=100)
plt.scatter(fare_per_mile, fare_per_minute)
plt.title('Fare per mile versus fare per minute (after cleaning)')
plt.xlabel('Fare/Mile')
plt.ylabel('Fare/Minute');

## Analysis Time!

In [ ]:
#Let's explore the tips and tolls columns
taxi_weather_clean.groupby(['payment_type'])['trip_miles'].describe()

> On average, trips paid with a credit card had longer distance, and therefore, longer duration and higher fare. However, I'm not going to use payment_type as a variable, since it's not relevant

In [ ]:
#Let's explore the distribution of variables of interest
plt.figure(figsize=(25, 5))

plt.subplot(1, 3, 1)
plt.hist(taxi_weather_clean['trip_miles'], bins=1000, color='m')
plt.title('Trip Length Histogram (in miles)')
plt.xlim(xmin=0, xmax=30)
plt.xlabel('Miles')
plt.ylabel('Frequency')

plt.subplot(1, 3, 2)
plt.hist(taxi_weather_clean['trip_seconds'], bins=1000, color='c')
plt.title('Trip Length Histogram (in seconds)')
plt.xlim(xmin=0, xmax=10000)
plt.xlabel('Seconds')
plt.ylabel('Frequency')

plt.subplot(1, 3, 3)
plt.hist(taxi_weather_clean['fare_with_tolls'], bins=5000, color='r')
plt.title('Trip Fare (in $)')
plt.xlim(xmin=0, xmax=100)
plt.xlabel('$')
plt.ylabel('Frequency');

### What if I log these variables?

In [ ]:
#Let's explore the distribution of variables of interest
plt.figure(figsize=(25, 5))

plt.subplot(1, 3, 1)
plt.hist(np.log(taxi_weather_clean['trip_miles']), bins=100, color='m')
plt.title('Trip Length Log Histogram (in miles)')
plt.xlabel('Miles')
plt.ylabel('Frequency')

plt.subplot(1, 3, 2)
plt.hist(np.log(taxi_weather_clean['trip_seconds']), bins=100, color='c')
plt.title('Trip Length Log Histogram (in seconds)')
plt.xlabel('Seconds')

plt.subplot(1, 3, 3)
plt.hist(np.log(taxi_weather_clean['fare_with_tolls']), bins=200, color='r')
plt.title('Trip Fare Log (in $)')
plt.xlabel('$');

> Good enough!

## Time Series fun

### Explore different cuts of time series

In [ ]:
taxi_weather_clean['pickup_date'] = taxi_weather_clean['trip_start_timestamp'].dt.date
taxi_weather_clean['pickup_time'] = taxi_weather_clean['trip_start_timestamp'].dt.time
# taxi_weather_clean['pickup_time_float'] = [t.mktime(start_time.timetuple()) for start_time in taxi_weather_clean['pickup_time']]
taxi_weather_clean['pickup_weekday'] = taxi_weather_clean['trip_start_timestamp'].dt.weekday      #Monday is 0

#This is specifically for the heatmap, not used in modelling or elsewhere
taxi_weather_clean['pickup_time_formatted'] = [time.strftime("%H:%M") for time in taxi_weather_clean['pickup_time']]    
taxi_weather_clean['pickup_weekday_name'] = taxi_weather_clean['trip_start_timestamp'].dt.weekday_name

In [ ]:
taxi_weather_clean.head()

### Fares by the date variables (day, weekday, time)?

In [ ]:
#Creating pandas series for days, time and weekday
avg_pickup_fare_per_mile_over_days = taxi_weather_clean.groupby(['pickup_date'])['fare_with_tolls'].sum() / taxi_weather_clean.groupby(['pickup_date'])['trip_miles'].sum()
avg_pickup_fare_per_mile_by_time = taxi_weather_clean.groupby(['pickup_time'])['fare_with_tolls'].sum() / taxi_weather_clean.groupby(['pickup_time'])['trip_miles'].sum()
avg_pickup_fare_per_mile_by_weekday = taxi_weather_clean.groupby(['pickup_weekday'])['fare_with_tolls'].sum() / taxi_weather_clean.groupby(['pickup_weekday'])['trip_miles'].sum()
                                    

In [ ]:
plt.figure(figsize=(25, 20))
plt.subplots_adjust(left=0.125, bottom=0.1, right=0.9, top=0.9,
                wspace=0.2, hspace=0.3)

avg_pickup_fare_per_mile_over_days_R52 = avg_pickup_fare_per_mile_over_days.rolling(52, center=True).mean()

plt.subplot(3, 1, 1)
avg_pickup_fare_per_mile_over_days_R52.plot()
plt.title('Average fare per mile over days')
plt.xlabel('Days')
plt.ylabel('Avg. Fare')

plt.subplot(3, 1, 2)
avg_pickup_fare_per_mile_by_time.plot(color='m')
plt.title('Average fare per mile over time')
plt.xlabel('Time')
plt.ylabel('Avg. Fare')

plt.subplot(3, 1, 3)
avg_pickup_fare_per_mile_by_weekday.plot(color='c',kind='bar')
plt.title('Average fare per mile by weekday')
plt.xlabel('Weekday (Monday = 0)')
plt.ylabel('Avg. Fare')
plt.ylim(3.4, 4.0);

> * Average fare across the year has been increasing from January to June, dips in summer then spikes again in October / November and dips again in December. It is strange that the average fare would drop so much in December, since it's an average.  
> * Weekday certainly has an impact on the fare, therefore I will be treating the weekday feature as a separate variable in the modelling  
> * There's something strange going on in the data between 5am and 7am. Seems like there were a few long distance (high fare) trips around this time. After noticing this, I change the measure from avg. fare to avg. fare per mile.
> * Looking at the first two charts, I certainly need to do some differencing to make my time series data more stationary.


### Zooming in on 5am-7am period

In [ ]:
taxi_weather_5_7am = taxi_weather_clean[(taxi_weather_clean['pickup_time'] >= dt.time(5,0,0)) &
                                         (taxi_weather_clean['pickup_time'] <= dt.time(7,0,0))]

# taxi_weather_5_7am = taxi_weather_5_7am.groupby(['pickup_time'])['fare_with_tolls'].max()

In [ ]:
plt.figure(figsize=(10, 5))
taxi_weather_5_7am['fare_with_tolls'].hist(bins=100);

In [ ]:
taxi_weather_5_7am[taxi_weather_5_7am['fare_with_tolls'] >= 1000.0]

> These records seem legitimate, since they have high mileage and hence high fare.

## Looking at the demand over time

In [ ]:
taxi_weather_clean.head(2)

In [ ]:
#Creating demand dataframe used in time series OLS
demand_over_time = taxi_weather_clean.groupby(['trip_start_timestamp'])['trip_start_timestamp'].count()

demand_over_days = taxi_weather_clean.groupby(['pickup_date'])['pickup_date'].count()
demand_by_15min = taxi_weather_clean.groupby(['pickup_time'])['pickup_time'].count()
demand_by_weekday = taxi_weather_clean.groupby(['pickup_weekday'])['pickup_weekday'].count()

In [ ]:
plt.figure(figsize=(25, 30))
plt.subplots_adjust(left=0.125, bottom=0.1, right=0.9, top=0.9,
                wspace=0.2, hspace=0.3)

plt.subplot(4, 1, 1)
demand_over_time.plot(color='c')
plt.title('Demand over time')
plt.xlabel('Date & Time')
plt.ylabel('Number of rides')

plt.subplot(4, 1, 2)
demand_over_days.plot()
plt.title('Demand over days')
plt.xlabel('Days')
plt.ylabel('Number of rides')

plt.subplot(4, 1, 3)
demand_by_15min.plot(color='m')
plt.title('Demand by 15 min intervals')
plt.xlabel('Time')
plt.ylabel('Number of rides')

plt.subplot(4, 1, 4)
demand_by_weekday.plot(color='c', kind='bar')
plt.title('Demand by weekday')
plt.xlabel('Weekday (Monday = 0)')
plt.ylabel('Number of rides');

> There is clearly a weekly pattern where Monday and Sunday seem to have the lowest demand (need to verify that, since it's a little surpising)  
> There is also a clear pattern in number of rides during the day, with spikes at peak hours, especially the afternoon peak and being slow early in the morning

### Fancy visualization :)

In [ ]:
time_map = taxi_weather_clean.pivot_table(index='pickup_time_formatted', columns=['pickup_weekday'], values='avg_temp_f', aggfunc='count')

In [ ]:
plt.figure()
plt.figure(figsize=(15,10))

ax = sns.heatmap(time_map, annot=False, xticklabels=['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'], cmap='coolwarm', cbar_kws={'label':'Taxi Demand'})
plt.title('Taxi demand by 15min time period by weekday')
ax.set(xlabel='Pickup Weekday', ylabel='Pickup Time');

# plt.savefig('Taxi_Demand_Heatmap.png', bbox_inches='tight');

### Slide formatting

In [ ]:
# fig_hm = plt.figure(figsize=(21, 11.7))
# fig_hm.patch.set_facecolor('#041F30')

# ax = sns.heatmap(time_map, annot=False, xticklabels=['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'], cmap='coolwarm')     # , cbar_kws={'label':'Taxi Demand'})
# plt.title('')                                     #Taxi demand by 15min time period by weekday
# ax.set(xlabel='', ylabel='');                     #Pickup Weekday     #Pickup Time
# ax.tick_params(colors='w', labelcolor='w')
# cax = plt.gcf().axes[-1]
# cax.tick_params(color='w', labelcolor='w')

# plt.savefig('Taxi_Demand_Heatmap_slides.png', facecolor=fig_hm.get_facecolor(), edgecolor=fig_hm.get_edgecolor());

### Understanding the stationarity of my timeseries data

#### Helper Function

In [ ]:
def test_stationarity(timeseries):   
    #Perform Dickey-Fuller test:
    print('Results of Dickey-Fuller Test:')
    timeseries.dropna(inplace=True)
    dftest = adfuller(timeseries, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value
    print(dfoutput)
    
test_stationarity(demand_by_15min)

> Since p-value is higher than 0.05, I cannot reject null hypothesis that the data is not stationare  
> The test result suggests that I need to do **differencing**

In [ ]:
print('Differencing over Days:')
print('0 diffs:', demand_over_time.std())
print('1 diffs:', demand_over_time.diff().std())
print('2 diffs:', demand_over_time.diff().diff().std())
print('3 diffs:', demand_over_time.diff().diff().diff().std())

> Differencing **once** seems to be the right call for both dates and times

In [ ]:
#So let's do it
demand_over_time_diffed = demand_over_time.diff()
demand_over_time_diffed = demand_over_time_diffed[1:]

In [ ]:
plt.figure(figsize=(25, 5))
demand_over_time_diffed.plot(color='c')
plt.title('Demand difference over time')
plt.xlabel('Date & Time')
plt.ylabel("Difference from demand 15 minutes ago");

In [ ]:
#What's the stationarity score like after differencing?
test_stationarity(demand_over_time_diffed)

> Much better now!

**Now check out how many lags to look back at (remember it's based on 15min time periods)**

In [ ]:
#Partial Autocorrelation plots will help to evaluate how many lags to include as features
print(plot_pacf(demand_over_time_diffed, lags=672, title='Partial Autocorrelation (672 lags)'))

> According to the Partial correlation plot, taking **7 first lags, 96th and 672th lags** is the way to go (last 1h 45min, same time yesterday and same time the week ago)

# NOW I'M READY TO START MODELLING... 
## ...please see the Time_Is_Money_Demand_Predictor notebook